In [1]:
from agent import Agent, autoChat
from questions import question
import threading
import time

/Users/jackson.makl@dataiku.com/agent/env/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Some weights of the model checkpoint at deepset/bert-base-cased-squad2 were not used when initializing BertForQuestionAnswering: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
class Model:
    def __init__(self):        
        self.agent = Agent()
        self.conscious = []
        self.subconscious = []
        self.threads = []
        self.lock = threading.Lock()  # Add a lock for thread safety

    def chat(self, prompt, web=False, rag=False, tokens=150, use_gpt=False,use_sub_gpt=True, iters=5):
        response = self.agent.chat(prompt=prompt,
                                  web=web,
                                  rag=rag,
                                  tokens=tokens,
                                  use_gpt=use_gpt,
                                  messages=self.conscious[-5:])

        generated_questions = question(response)
        
        # Create a semaphore that allows max 3 concurrent threads
        max_threads = 3
        semaphore = threading.Semaphore(max_threads)

        # Clear previous threads
        self.threads = []

        # Define the worker function correctly
        def thread_worker(q, semaphore):
            with semaphore:  # Acquire the semaphore
                result = autoChat(q, None, iters, 75, True, False, use_sub_gpt)
                # Safely add to subconscious using lock
                with self.lock:
                    self.subconscious.append(result)

        # Create threads with the properly defined worker
        for q in generated_questions:
            thread = threading.Thread(
                target=thread_worker,
                args=(q, semaphore)
            )
            self.threads.append(thread)
            thread.start()  # Start the thread immediately

        self.conscious.append({"role": "user", "content": prompt})
        self.conscious.append({"role": "assistant", "content": response})  # Changed "user" to "assistant"
        
        return response

In [3]:
m=Model()

In [ ]:
m.chat("What Challenges Does Airbnb Face?",web=False,rag=True,tokens=150,use_gpt=True,iters=5)

/Users/jackson.makl@dataiku.com/agent/rag.py:31: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  self.embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
100%|██████████| 4/4 [00:00<00:00, 1097.99it/s]


USING CHAT GPT 4
Agent searching the web.....
Researching: 
    What is **What Challenges Does?
    
Agent searching the web.....
Researching: 
    How does **Pricing Inequity and?
    
Agent searching the web.....
Researching: 
    Why is However, in cities?
    


"**What Challenges Does Airbnb Face?**\n\nAirbnb faces several significant challenges:\n\n1. **Pricing Inequity and Socioeconomic Segregation:**  \n   The company's pricing model suggests prices based on similar properties in the area, considering location, amenities, and past bookings. However, in cities like New York where neighborhoods are often segregated along racial and ethnic lines (e.g., Black, Colombian, Russian, and Italian neighborhoods), this automated pricing may perpetuate existing inequalities. Even if race is not directly a factor in the algorithm, the model may reflect and reinforce socio-economic disparities tied to historical segregation.\n\n2. **Indirect Racial Bias:**  \n   Black hosts may end up charging less not because of direct discrimination, but because their properties are located in historically underserved neighborhoods. This creates indirect inequity, where hosts from certain backgrounds systematically earn less even if the system doesn't explicitly use r

Found 5 total results
After deduplication: 5 results
Fetching additional content from URLs...
Found 5 total results
After deduplication: 5 results
Fetching additional content from URLs...
Found 15 total results
After deduplication: 14 results
Fetching additional content from URLs...
Researching: types of inequity effects on behavior health fairness access to resources
Researching: examples of challenges in daily life and current events
Found 10 total results
After deduplication: 10 results
Fetching additional content from URLs...
Found 5 total results
After deduplication: 5 results
Fetching additional content from URLs...
Researching: city definitions administrative vs actual urban areas population measurement China UK
Found 5 total results
After deduplication: 5 results
Fetching additional content from URLs...
USING CHAT GPT 4
USING CHAT GPT 4
Researching: 
        Instructions:
          - Engage naturally as if you are having a real conversation.
          
          - If you need o

In [5]:
m.conscious

[{'role': 'user', 'content': 'What Challenges Does Airbnb Face?'},
 {'role': 'assistant',
  'content': "**What Challenges Does Airbnb Face?**\n\nAirbnb faces several significant challenges:\n\n1. **Pricing Inequity and Socioeconomic Segregation:**  \n   The company's pricing model suggests prices based on similar properties in the area, considering location, amenities, and past bookings. However, in cities like New York where neighborhoods are often segregated along racial and ethnic lines (e.g., Black, Colombian, Russian, and Italian neighborhoods), this automated pricing may perpetuate existing inequalities. Even if race is not directly a factor in the algorithm, the model may reflect and reinforce socio-economic disparities tied to historical segregation.\n\n2. **Indirect Racial Bias:**  \n   Black hosts may end up charging less not because of direct discrimination, but because their properties are located in historically underserved neighborhoods. This creates indirect inequity, wh

In [9]:
m.subconscious

[]